# **IMPORT Libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

# **LOAD CIFAR10 dataset**

In [2]:
# Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

170498071/170498071 [==============================] - 9s 0us/step


# **Flatten the images**

In [3]:
# Flatten the images for logistic regression
X_train_flat = X_train.reshape((X_train.shape[0], -1))
X_test_flat = X_test.reshape((X_test.shape[0], -1))

# **Normalize pixel values to be in the range [0, 1]**

In [4]:
# Normalize pixel values to be in the range [0, 1]
X_train_flat = X_train_flat / 255.0
X_test_flat = X_test_flat / 255.0

# **Initialize the StandardScaler**

In [5]:
# Initialize the StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_flat)
X_test_scaled = scaler.transform(X_test_flat)

# **Define the parameter grid for randomized search**

In [6]:
# Define the parameter grid for randomized search
param_dist = {
    'hidden_layer_sizes': [256],
    'activation': ['relu'],
    'learning_rate_init': [0.0001],
    'alpha': [3.76],
    'batch_size': [64],
    'max_iter': [10],
    'solver': ['sgd'],
}


# **MLPClassifier**

In [7]:
# Initialize the MLPClassifier
mlp = MLPClassifier(random_state=42)

# Initialize the RandomizedSearchCV
random_search = RandomizedSearchCV(
    mlp,
    param_distributions=param_dist,
    n_iter=10,  # Number of parameter settings that are sampled
    scoring='accuracy',  # You can choose a different scoring metric
    cv=5,  # Number of cross-validation folds
    n_jobs=-1,  # Use all available CPU cores
    random_state=42
)

# Fit the RandomizedSearchCV to the data
random_search.fit(X_train_scaled, y_train.ravel())


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


RandomizedSearchCV(cv=5, estimator=MLPClassifier(random_state=42), n_jobs=-1,
                   param_distributions={'activation': ['relu'], 'alpha': [3.76],
                                        'batch_size': [64],
                                        'hidden_layer_sizes': [256],
                                        'learning_rate_init': [0.0001],
                                        'max_iter': [10], 'solver': ['sgd']},
                   random_state=42, scoring='accuracy')

In [8]:
# Get the best parameters and the best model
best_params = random_search.best_params_
best_mlp = random_search.best_estimator_

# Make predictions on the test set
y_pred_mlp = best_mlp.predict(X_test_scaled)

# Calculate accuracy
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)
print(f"MLP Accuracy: {accuracy_mlp}")

# Print the best hyperparameters
print("Best Hyperparameters:")
print(best_params)

MLP Accuracy: 0.457
Best Hyperparameters:
{'solver': 'sgd', 'max_iter': 10, 'learning_rate_init': 0.0001, 'hidden_layer_sizes': 256, 'batch_size': 64, 'alpha': 3.76, 'activation': 'relu'}
